In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from py2neo import Graph
import numpy as np

In [2]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

In [3]:
#Transform raw data to real pandas data
def identity(row):
    result = {}
    for val in row.values:
        result = {**result, **dict(val)}
    return result

In [4]:
def write_relation(X, file, relation):
    file.write(f"{X[0]}\t{relation}\t{X[1]}\n")

In [5]:
def GTE_prep_df(value, valueToCompareTo):
    return value >= valueToCompareTo

def LTE_prep_df(value, valueToCompareTo):
    return value <= valueToCompareTo

def GT_prep_df(value, valueToCompareTo):
    return value > valueToCompareTo

def LT_prep_df(value, valueToCompareTo):
    return value < valueToCompareTo

In [6]:
def write_features(X, file, object_name, description):
    for feature in X.drop("id").index:
#         file.write(f"{X['id']}\t{object_name}_{feature}\t{X[feature]}\n")
        if feature in description.columns:
            for threshold in description.index:
                file.write(f"{X['id']}\t{object_name}_{feature}_GTE_{threshold}%\t{GTE_prep_df(X[feature], description[feature].loc[threshold])}\n")
                file.write(f"{X['id']}\t{object_name}_{feature}_LTE_{threshold}%\t{LTE_prep_df(X[feature], description[feature].loc[threshold])}\n")
        else:
            file.write(f"{X['id']}\t{object_name}_{feature}\t{X[feature]}\n")

In [7]:
def descriptionDiffThresholds(data, percents):
    thresholds_Dic =  {}
    types =  tract_demo.dtypes
    for column in data.columns:
        if types[column] != "object":
            serie = data[column]
            thresholds_Dic[column] = serie.quantile(percents)
        
    return pd.DataFrame.from_dict(thresholds_Dic)

# 1 First decribe all the links between objects

Such as tract - [:HAS_demo] - Demo

In [8]:
links = [("(t:Tract)-[:HAS_SOCIAL_CHARACTERISTICS]-(s:SocialCharacteristics)", "t.id, s.id", "has_social_charac"),
        ]
#         ("(s:SocialCharacteristics)-[:HAS_COMPUTERS]-(c:Computers)", "s.id, c.id", "has_computers"),
#          ("(t:Tract)-[:HAS_ECONOMICS]-(e:Economics)", "t.id, e.id", "has_economics"), 
#         ("(e:Economics)-[:HAS_POVERTY]-(p:Poverty)", "e.id, p.id", "has_economics")]

#("(t:Tract)-[:HAS_DEMOGRAPHICS]-(d:Demographics)", "t.id, d.id", "has_demographics")

In [9]:
dict_keep_link = {}

In [10]:
f = open("Covid.tsv", "w")

for link in links:
    print(link[2])
    query = f"""
        MATCH {link[0]}
        RETURN {link[1]}
        """
    raw_answer = graph.run(query)
    tract_demo = raw_answer.to_data_frame()
    tract_demo.apply(write_relation, axis=1, relation=link[2], file=f)
    dict_keep_link[link] = tract_demo
    
f.close()

has_social_charac


# 2 Read the features hidden in the objects

In [11]:
thresholds = np.arange(0.25,1,0.25) #[0.25, 0.50, 0.75]
thresholds_str = ["-"+(str(int(i*100))) for i in thresholds]

In [12]:
objects_features = ["Tract", "Demographics"]#, "Poverty", "SocialCharacteristics", "Computers"] #"Demographics"

In [13]:
dict_keep_features = {}

In [14]:
f = open("Covid.tsv", "a")

for o_f in objects_features:
    print(o_f)
    print("\tStart Query")
    query = f"""
        MATCH (o:{o_f})
        RETURN o
        """
    raw_answer = graph.run(query)
    print("\tQuery finished")
    tract_demo = raw_answer.to_data_frame()
    tract_demo = tract_demo.apply(identity, axis=1).apply(pd.Series)
    description = descriptionDiffThresholds(tract_demo, thresholds)
    dict_keep_features[o_f] = tract_demo
    print("\tWrite the data")
    tract_demo.apply(write_features, axis=1, file=f, object_name=o_f, description=description)
    
f.close()

Tract
	Start Query
	Query finished
	Write the data
Demographics
	Start Query
	Query finished
	Write the data


# 3 Time to merge some stuff

In [135]:
dict_keep_features['Tract'].merge(right=dict_keep_link[("(t:Tract)-[:HAS_SOCIAL_CHARACTERISTICS]-(s:SocialCharacteristics)", "t.id, s.id", "has_social_charac")], left_on="id", right_on="t.id").merge(right=dict_keep_features["SocialCharacteristics"], left_on='s.id', right_on='id')

,name_x,id_x,t.id,s.id,stateFips,countyFips,aggregationLevel,name_y,id_y,source,postalCode,tract
0,36103158607,tract36103158607,tract36103158607,ACSDP5Y2018.DP02-36103158607,NaN,NaN,Tract,SocialCharacteristics-36103158607,ACSDP5Y2018.DP02-36103158607,American Community Survey 5 year,NaN,36103158607
1,25013813207,tract25013813207,tract25013813207,ACSDP5Y2018.DP02-25013813207,NaN,NaN,Tract,SocialCharacteristics-25013813207,ACSDP5Y2018.DP02-25013813207,American Community Survey 5 year,NaN,25013813207
2,25013813206,tract25013813206,tract25013813206,ACSDP5Y2018.DP02-25013813206,NaN,NaN,Tract,SocialCharacteristics-25013813206,ACSDP5Y2018.DP02-25013813206,American Community Survey 5 year,NaN,25013813206
3,25013813205,tract25013813205,tract25013813205,ACSDP5Y2018.DP02-25013813205,NaN,NaN,Tract,SocialCharacteristics-25013813205,ACSDP5Y2018.DP02-25013813205,American Community Survey 5 year,NaN,25013813205
4,25013813204,tract25013813204,tract25013813204,ACSDP5Y2018.DP02-25013813204,NaN,NaN,Tract,SocialCharacteristics-25013813204,ACSDP5Y2018.DP02-25013813204,American Community Survey 5 year,NaN,25013813204
...,...,...,...,...,...,...,...,...,...,...,...,...
73051,55117990000,tract55117990000,tract55117990000,ACSDP5Y2018.DP02-55117990000,NaN,NaN,Tract,SocialCharacteristics-55117990000,ACSDP5Y2018.DP02-55117990000,American Community Survey 5 year,NaN,55117990000
73052,55101980000,tract55101980000,tract55101980000,ACSDP5Y2018.DP02-55101980000,NaN,NaN,Tract,SocialCharacteristics-55101980000,ACSDP5Y2018.DP02-55101980000,American Community Survey 5 year,NaN,55101980000
73053,55101990000,tract55101990000,tract55101990000,ACSDP5Y2018.DP02-55101990000,NaN,NaN,Tract,SocialCharacteristics-55101990000,ACSDP5Y2018.DP02-55101990000,American Community Survey 5 year,NaN,55101990000
73054,55031990000,tract55031990000,tract55031990000,ACSDP5Y2018.DP02-55031990000,NaN,NaN,Tract,SocialCharacteristics-55031990000,ACSDP5Y2018.DP02-55031990000,American Community Survey 5 year,NaN,55031990000


In [136]:
dict_keep_features["Tract"]

,name,id
0,36103158607,tract36103158607
1,72001956800,tract72001956800
2,72113071700,tract72113071700
3,72001956600,tract72001956600
4,72001956700,tract72001956700
...,...,...
73902,55117990000,tract55117990000
73903,55101980000,tract55101980000
73904,55101990000,tract55101990000
73905,55031990000,tract55031990000


In [137]:
dict_keep_features["SocialCharacteristics"]

,stateFips,countyFips,aggregationLevel,name,id,source,postalCode,tract
0,02,195,Admin2,SocialCharacteristics-02-195,ACSDP5Y2018.DP02-02-195,American Community Survey 5 year,NaN,NaN
1,02,158,Admin2,SocialCharacteristics-02-158,ACSDP5Y2018.DP02-02-158,American Community Survey 5 year,NaN,NaN
2,04,013,Admin2,SocialCharacteristics-04-013,ACSDP5Y2018.DP02-04-013,American Community Survey 5 year,NaN,NaN
3,04,009,Admin2,SocialCharacteristics-04-009,ACSDP5Y2018.DP02-04-009,American Community Survey 5 year,NaN,NaN
4,04,023,Admin2,SocialCharacteristics-04-023,ACSDP5Y2018.DP02-04-023,American Community Survey 5 year,NaN,NaN
...,...,...,...,...,...,...,...,...
109313,NaN,NaN,Tract,SocialCharacteristics-56015958000,ACSDP5Y2018.DP02-56015958000,American Community Survey 5 year,NaN,56015958000
109314,NaN,NaN,Tract,SocialCharacteristics-56015957700,ACSDP5Y2018.DP02-56015957700,American Community Survey 5 year,NaN,56015957700
109315,NaN,NaN,Tract,SocialCharacteristics-56015957900,ACSDP5Y2018.DP02-56015957900,American Community Survey 5 year,NaN,56015957900
109316,NaN,NaN,Tract,SocialCharacteristics-56003962600,ACSDP5Y2018.DP02-56003962600,American Community Survey 5 year,NaN,56003962600


In [ ]:
withBroadbandInternet withBroadbandInternetPct withComputer